# 이미지 식별 머신을 위한 데이터를 준비한다.

## 필요한 라이브러리를 불러 온다.

In [2]:
# 데이터 플로팅 라이브러리
import matplotlib.pyplot as plt
# 숫자 처리 라이브러리
import numpy as np
# 딥러닝을 위한 파이토치 라이브러리
import torch
from torch import nn, optim
# 토치비전 라이브러리
import torchvision
from torchvision import datasets, transforms, models
# 이미지 처리 라이브러리 (PIL, pillow)
from PIL import Image
# 주피터 노트북에서 plot이 보이도록 설정
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 데이터 디렉토리, 분할 비율, 변환 방법을 설정한다.

In [3]:
# 이미지 데이터가 있는 디렉토리와 데이터 세트 분할 비율(valid_size)을 정한다.
data_dir = '../data'
valid_size=0.2

# 이미지 데이터를 ResNet50에서 다룰 수 있도록 변환시키는 방법을 정한다. (t_transforms)
t_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.Resize(224),
    transforms.ToTensor()
])
# convert image size to 224x224 for ResNet50 after crop


### (확인) 변환 방법을 출력하여 확인해 본다.

In [4]:
# 설정한 이미지 데이터 변환 방법을 출력하여 확인한다.
print(t_transforms)

Compose(
    RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
    Resize(size=224, interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
)


## 데이터를 로딩 함수를 작성한다.

### (연습) trainloader와 testloader를 만들어 본다.

#### 1. 학습 데이터 세트 및 테스트 데이터 세트의 디렉토리 및 변환 방식을 지정한다.

In [5]:
# datasets.ImageFolder를 사용해서 학습 데이터(train_data)와 테스트 데이터(test_data)를 만든다.
# make train_data and test_data using datasets.ImageFolder
train_data = datasets.ImageFolder(data_dir, transform=t_transforms)
test_data = datasets.ImageFolder(data_dir, transform=t_transforms)

# 학습 데이터의 형식을 확인한다.
print(train_data)
# 학습 데이터와 테스트 데이터의 길이를 확인한다.
print(len(train_data), len(test_data))

Dataset ImageFolder
    Number of datapoints: 155
    Root location: ../data
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
               Resize(size=224, interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
155 155


#### 2. 데이터세트를 섞기 위해, 우선 인덱스를 만들어 랜덤하게 섞는다.

In [6]:
# train_data 사이즈만큼의 정수값을 갖는 인덱스 리스트(indices)를 만들고 확인한다.
num_train = len(train_data)
indices = list(range(num_train))
print(indices)

# 인덱스 리스트를 랜덤하게 섞고 확인한다.
np.random.shuffle(indices)
print(indices)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154]
[95, 2, 41, 128, 143, 93, 74, 50, 146, 22, 123, 26, 43, 87, 40, 115, 15, 52, 90, 56, 28, 102, 9, 111, 1, 119, 16, 139, 108, 7, 131, 77, 85, 136, 18, 51, 120, 118, 125, 63, 106, 21, 135, 32, 44, 89, 78, 134, 49, 65, 12, 33, 110, 47, 153, 69, 104, 88, 29, 147, 57, 132, 5, 91, 8, 122, 149, 23, 96, 141, 62, 142, 55, 20, 27, 126, 61, 98,

#### 3. 분할 비율(valid_size)에 따른 지점의 인덱스 값(split)을 계산한다.

In [7]:
# 분할 비율(valid_size)에 해당하는 인덱스를 계산하고 확인해 본다.
split = int(np.floor(num_train * valid_size))
print(split)

31


#### 4. split을 기준으로 학습 데이터 인덱스 리스트와 테스트 인덱스 리스트로 나눈다.

In [8]:
# 학습 데이터 인덱스 리스트 및 테스트 인덱스 리스트를 만들고 확인해 본다.
train_idx, test_idx = indices[split:], indices[:split]

print(train_idx)
print(test_idx)

[77, 85, 136, 18, 51, 120, 118, 125, 63, 106, 21, 135, 32, 44, 89, 78, 134, 49, 65, 12, 33, 110, 47, 153, 69, 104, 88, 29, 147, 57, 132, 5, 91, 8, 122, 149, 23, 96, 141, 62, 142, 55, 20, 27, 126, 61, 98, 112, 121, 71, 127, 6, 101, 133, 116, 80, 117, 151, 25, 45, 68, 105, 114, 11, 72, 37, 81, 19, 53, 94, 67, 84, 75, 97, 60, 144, 38, 83, 35, 79, 13, 31, 54, 17, 82, 129, 70, 36, 76, 14, 150, 66, 73, 138, 124, 130, 10, 0, 3, 107, 109, 24, 34, 46, 152, 92, 148, 103, 59, 42, 140, 58, 39, 4, 30, 99, 64, 113, 100, 48, 137, 154, 145, 86]
[95, 2, 41, 128, 143, 93, 74, 50, 146, 22, 123, 26, 43, 87, 40, 115, 15, 52, 90, 56, 28, 102, 9, 111, 1, 119, 16, 139, 108, 7, 131]


#### 5. 데이터 세트들의 샘플러 및 로더를 만들고 확인한다.

In [11]:
# 데이터 샘플링 방식(SubsetRandomSampler)을 지정한다
from torch.utils.data.sampler import SubsetRandomSampler
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)

# 데이터 로딩을 위한 loader를 만든다. (sampler, 배치 사이즈 등 지정)
trainloader = torch.utils.data.DataLoader(train_data, sampler = train_sampler, batch_size=16)
testloader = torch.utils.data.DataLoader(test_data, sampler = test_sampler, batch_size = 16)

# 학습 loader와 테스트 loader의 class들을 출력하여 확인한다.
print(trainloader.dataset.classes)
print(testloader.dataset.classes)

['Basalt', 'Highland']
['Basalt', 'Highland']


### 코드들을 묶어서 load_split_train_test() 함수를 만든다.

In [12]:
# 위의 코드들을 묶어서 load_split_train_test() 함수를 만든다. (입력 : 데이터 디렉토리, 분할 비율) (출력 : 학습 데이터 로더, 테스트 데이터 로더)
def load_split_train_test(data_dir, valid_size):
    t_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.Resize(224),
        transforms.ToTensor()
    ])
    
    train_data = datasets.ImageFolder(data_dir, transform=t_transforms)
    test_data = datasets.ImageFolder(data_dir, transform=t_transforms)

    num_train = len(train_data)
    indices = list(range(num_train))

    np.random.shuffle(indices)
    split = int(np.floor(num_train * valid_size))
    train_idx, test_idx = indices[split:], indices[:split]


    from torch.utils.data.sampler import SubsetRandomSampler
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    trainloader = torch.utils.data.DataLoader(train_data, sampler = train_sampler, batch_size=16)
    testloader = torch.utils.data.DataLoader(test_data, sampler = test_sampler, batch_size = 16)

    return trainloader, testloader

### load_split_train_test() 함수를 이용하여 trainloader, testloader를 생성한다.

In [13]:
# load_split_train_test() 함수를 이용하여 trainloader와 testloader를 만들고 확인한다.
trainloader, testloader = load_split_train_test(data_dir, valid_size=0.2)

print(trainloader.dataset.classes)
print(testloader.dataset.classes)

['Basalt', 'Highland']
['Basalt', 'Highland']


## 이미지 데이터 샘플들을 살펴본다.

### 임의의 데이터를 로딩한 후 이미지와 레이블을 반환하는 get_random_images() 함수를 만든다.

### 임의 선택한 이미지를 표시해 본다.

In [ ]:
# 5개의 이미지와 레이블을 랜덤하게 가져온다.

# 픽셀 배열을 PIL 형식의 이미지로 변환하고 이미지 크기를 지정한다.

# 학습 데이터의 class 리스트를 얻는다.

# 이미지를 표시하기 위한 설정을 한다.

# 주피터 노트북에 이미지를 표시한다.



# ResNet50 모델을 가져와 FCL(Fully Connected Layer)을 수정한다.

## Compute device를 정한다(CPU or GPU)

In [ ]:
# compute device를 정하고 확인한다.


## 사전학습된 ResNet50 모델을 지정한다.

In [ ]:
# resnet50 모델을 pretrained=True로 설정한다.


### (확인) 수정 전의 ResNet50 모델을 확인해 본다. 

In [ ]:
print(model)

## FCL을 수정한다.(뉴런 구축, 신경망 연결, FCL의 layer 설정 등)

In [ ]:
# 모든 신경망 구축 : 전이학습을 위해 모델의 가중치를 freeze 한다.
    
# 뉴런들을 연결하여 신경망을 생성한다.

# q: explain the above code
# a: 2048개의 입력을 받아 512개의 출력을 내고, ReLU 함수를 거쳐 0.2의 확률로 Dropout을 적용한다.
# 512개의 입력을 받아 2개의 출력을 내고, LogSoftmax 함수를 거쳐 1차원으로 변환한다.
# 1차원으로 변환된 출력을 갖는 신경망을 생성한다.

# 손실함수를 Cross entropy loss 함수로 지정한다.

# why
# optimizer를 Adam으로 지정한다.
# what is Adam
#

# 신경망을 compute device로 보낸다.

# 종료 여부를 출력한다.



### (확인) FCL을 확인해 본다.

# 모델의 FCL을 학습시키고 테스트 한다.

## 모델 학습/검증을 위한 변수를 설정한다.

In [ ]:
# 에폭 및 출력 간격을 설정한다.

# 손실 변수들을 초기화 한다.

# 현재의 학습 단계를 표현하는 steps 변수를 0으로 초기화 한다.



## 설정한 에폭만큼 모델을 학습시키며 검증/평가 한다.

In [ ]:
# 설정한 회수만큼 학습 후 테스트 및 평가해 본다.

    # 에폭을 count 한다.

    # trainloader로부터 모든 이미지와 레이블을 로드한다.

        # 학습 단계를 count 하고 출력한다.

        # 입력 데이터(이미지, 레이블)를 device로 보낸다.

        # 기존에 학습된 gradient 값을 초기화 한다.(이전에 학습한 값이 영향을 주지 않도록 함)

        # 입력 데이터로 순전파를 수행하고 로그 확률을 얻는다.

        # 손실(loss) 값들을 계산한다.

        # 손실값을 이용하여 gradient를 update한다.

        # gradient를 이용하여 설정된 optimizer로 파라미터를 update한다.

        # 손실값을 누적/계산한다.

        # 학습 단계 5회마다 모델을 테스트/평가 한다.

            # 손실과 정확도 변수를 초기화 한다.

            # 모델 평가 모드로 전환한다.

            # 모델 평가 시 gradient를 계산하지 않도록 한다.

                # testloader로부터 모든 이미지와 레이블을 로드한다.

                    # 입력 데이터(이미지, 레이블)를 device로 보낸다.
 
                    # 입력 데이터로 순전파를 수행하고 로그 확률을 얻는다.

                    # 손실(loss) 값들을 계산한다.

                    # 손실값을 누적시킨다.

                    # 로그 확률로부터 진짜 확률값을 계산한다.

                    # 가장 큰 확률값과 class를 얻는다. (topk : k번째로 큰 값)

                    # 레이블들을 top_class와 동일한 형태로 바꾼 후 같은 값들을 얻는다.

                    # equals를 float 텐서로 바꾼 후 평균 정확도를 누적/계산한다.

            # 학습 손실값과 테스트 손실값을 추가한다.

            # 학습 손실값, 테스트 손실값, 테스트 정확도를 출력한다.

            # running_loss 값을 초기화 한다.

            # 모델 학습 모드로 전환한다.



### (확인) 학습 손실값과 테스트 손실값을 그래프로 확인한다.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'



# in this graph, what is x-axis? y-axis?
# x-axis: epoch
# y-axis: loss

# 학습/테스트 완료된 모델을 저장한다.

In [ ]:
# 추후 로드하여 사용할 수 있도록 학습/테스트 완료된 모델을 저장한다.


# 완성된 모델을 사용하여 예측한다.

## 저장한 모델을 불러온다.

In [ ]:
# 저장한 모델을 불러온다.



### (확인) 불러온 모델을 확인해 본다.

## 이미지 예측을 위해 predict_image() 함수를 만든다.

## 5개의 이미지를 임의로 가져와 예측해 본다.

In [ ]:
# 모델 평가 모드로 전환한다.

# 5개의 이미지를 랜덤하게 가져온 후 PIL 형식 변환, 표시할 이미지 크기를 설정한다.

# 데이터의 class 목록을 얻는다.


# 5개의 이미지에 대해 loop를 수행한다.

    # 각 이미지에 대해 class를 예측한다.


    # 이미지 아래에 class를 표시하도록 설정한다.

# 레이블이 추가된 이미지를 보여준다.

